##### MAIN OVERVIEW:
##### Written 2023/06 by Caroline Goodman and refined by Christina Lebonville 2023/07.
##### Last updated: 2023/08/10 CL

##### REQUIRES KERNEL "firstEnv"
##### For Determining Real Bouts from Fiber Photometry Videos using DeepEthogram - Integrates DEG Predictions and FP Output
    For each workbook:
        - find file containing labels that correspond to workbook
        - determine % of consumatory frames according to DEG
        - update dataframe to include % consumatory frames
        - save updated dataframe to updated_workbooks folder

##### Import necessary packages.

In [84]:
import pandas as pd
import os

##### get_consumatory_percentage(workbook, labels):
    - takes each bout period and converts it to DEG frames. Then computes percent of DEG labels during this period that are labeled consumatory
##### get_corresponding_labels_file(file_path):
    - finds the file containing the DEG predictions that correspond to the TDT Fiber data
    - For example:
        - 20221202CL20A6B5quinineresultsB.xlsx corresponds to --> 20221201CL20A6B5quinine_Cam1B_predictions.xlsx
##### generate_new_file(labels_path, workbook_df):
    - imports the predictions into a dataframe based on frame numbers and runs the update_workbook function
##### update_workbook(workbook_df, percentages, file_path):
    - adds the column to the TDT Fiber data xlsx to report % consumatory within bout
    - saves updated workbook to updated_workbooks folder    

In [85]:
def get_consumatory_percentage(workbook, labels):
    bout_percentages = []
    # for each bout
    # get start and end frame
    for start, duration in zip(workbook[workbook.columns[0]], workbook[workbook.columns[1]]):
        end = start+duration
        start = round((start+15)*10)
        end = round((end+15)*10)
        consum = 0
        total = 0
        error = []
        #print(start, end)
        # for each frame in the bout
        for frame_number in range(start, end+1):
            #print(frame_number)
            if frame_number in labels[labels.columns[0]]:
                if labels.loc[frame_number, 'C'] == 1:
                    #print('C')
                    consum += 1
                    total += 1
                else:
                    #print('NC')
                    total += 1
        if total == 0:
            print("WARNING: Some frames in workbook were not found in the predictions file")
            percent_consumatory = "error" 
            bout_percentages.append(percent_consumatory) 
        else: 
            #print(total)
            percent_consumatory = consum/total
            #print(consum, "/", total, "=", percent_consumatory)
            bout_percentages.append(percent_consumatory)  
    return bout_percentages
           
def get_corresponding_labels_file(file_path):
    cam = file_path[-6]
    file_name_chunk = file_path[:-13]
    print("Looking for labels...")
    return f"Lebonville_{file_name_chunk}_Cam1{cam}_predictions.csv"

def update_workbook(workbook_df, percentages, file_path):
    ## Feature Predictions
    #output_path = os.path.join(r"\\mulhollnas.thurmond-gazes.musc.edu\mulhollnas\Christina_Lebonville\TrueBout_2023_CL8_CL20_DEG_Model\updated_workbooks\feature_predictions", file_path)
    ##Sequence Predictions
    #output_path = os.path.join(r"\\mulhollnas.thurmond-gazes.musc.edu\mulhollnas\Christina_Lebonville\TrueBout_2023_CL8_CL20_DEG_Model\updated_workbooks\sequence_predictions", file_path)
    output_path = os.path.join(r"\\mulhollnas.thurmond-gazes.musc.edu\mulhollnas\Christina_Lebonville\TrueBout_2023_CL8_CL20_DEG_Model\Updated_CL8_Workbooks", file_path)
    
    # Add column denoting % Consumatory
    workbook_df['% of C'] = percentages
    
    # Create new Excel file with updated 'C or NC' column; saved to the updated_workbooks folder
    workbook_df.to_excel(output_path, index=False, engine='xlsxwriter')
    print("-----------> updated workbook exported to:", output_path)

def generate_new_file(labels_path, workbook_df, file):
    
    #create data frame for labels file
    labels_df = pd.read_csv(labels_path)
    
    # get frame ranges
    consumatory_percentages = get_consumatory_percentage(workbook_df, labels_df)
    
    # update and export the excel with % consumatory
    update_workbook(workbook_df, consumatory_percentages, file)

def main():
    
    #workbooks_directory = r"\\mulhollnas.thurmond-gazes.musc.edu\mulhollnas\Christina_Lebonville\TrueBout_2023_CL8_CL20_DEG_Model" #This is the folder containing FP output files
    workbooks_directory = r"\\mulhollnas.thurmond-gazes.musc.edu\mulhollnas\Christina_Lebonville\TrueBout_2023_CL8_CL20_DEG_Model\CL8_Data"
    
    ##Feature Predictions
    #labels_directory = r"\\mulhollnas.thurmond-gazes.musc.edu\mulhollnas\Christina_Lebonville\TrueBout_2023_CL8_CL20_DEG_Model\PerformanceCheck\feature_predictions" #This is the DEG prediction folder (.csv)
    ##Sequence Predictions
    #labels_directory = r"\\mulhollnas.thurmond-gazes.musc.edu\mulhollnas\Christina_Lebonville\TrueBout_2023_CL8_CL20_DEG_Model\PerformanceCheck\sequence_predictions" #This is the DEG prediction folder (.csv)
    labels_directory = r"\\mulhollnas.thurmond-gazes.musc.edu\mulhollnas\Christina_Lebonville\TrueBout_2023_CL8_CL20_DEG_Model\Gen4-5_Predictions"
    
    for file in os.listdir(workbooks_directory):
        # add individual file to file path
        file_path = os.path.join(workbooks_directory, file)
        print(file)
        # create data frame for particular file path
        workbook_df = pd.read_excel(file_path, 'Sheet4')
        
        # if bouts were found
        if 'ABoutStarts' in workbook_df.columns or 'BBoutStarts' in workbook_df.columns:
            
            # get labels file for particular file path
            os.chdir(labels_directory)
            labels_path = get_corresponding_labels_file(file)
            
            # continue only if a labels file is found in the directory
            if os.path.isfile(labels_path):
                generate_new_file(labels_path, workbook_df, file)
            else:
                print("No Labels File Found")

In [86]:
main()

20201026CL8A1B8resultsA.xlsx
Looking for labels...
-----------> updated workbook exported to: \\mulhollnas.thurmond-gazes.musc.edu\mulhollnas\Christina_Lebonville\TrueBout_2023_CL8_CL20_DEG_Model\Updated_CL8_Workbooks\20201026CL8A1B8resultsA.xlsx
20201026CL8A1B8resultsB.xlsx
Looking for labels...
-----------> updated workbook exported to: \\mulhollnas.thurmond-gazes.musc.edu\mulhollnas\Christina_Lebonville\TrueBout_2023_CL8_CL20_DEG_Model\Updated_CL8_Workbooks\20201026CL8A1B8resultsB.xlsx
20201028CL8A4B10resultsA.xlsx
Looking for labels...
-----------> updated workbook exported to: \\mulhollnas.thurmond-gazes.musc.edu\mulhollnas\Christina_Lebonville\TrueBout_2023_CL8_CL20_DEG_Model\Updated_CL8_Workbooks\20201028CL8A4B10resultsA.xlsx
20201028CL8A4B10resultsB.xlsx
Looking for labels...
-----------> updated workbook exported to: \\mulhollnas.thurmond-gazes.musc.edu\mulhollnas\Christina_Lebonville\TrueBout_2023_CL8_CL20_DEG_Model\Updated_CL8_Workbooks\20201028CL8A4B10resultsB.xlsx
20201029

##### CHANGELOG ######
##### 2023.08.10 - CL added code to deal with errors where frames were not found in the prediction files - meaning the videos were shorter than the fiber data for some reason (likely crashing). These bouts will now show "error" in their %C column and a warning will be printed.